In [64]:
import torch

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

import random
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer

from detectron2.utils.visualizer import ColorMode
import glob

from detectron2.data.datasets import register_coco_instances

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [65]:
register_coco_instances("radar_dynamic_train", {}, "/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/train/_annotations.coco.json", "/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/train")
register_coco_instances("radar_dynamic_test", {}, "/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/_annotations.coco.json", "/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test")


In [80]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("radar_dynamic_train",)
cfg.DATASETS.TEST = ("radar_dynamic_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000   # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.SOLVER.LOG_PERIOD = 1  # Log every iteration
## Note: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[01/18 13:40:54 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[01/18 13:40:54 d2.engine.train_loop]: Starting training from iteration 0
[01/18 13:41:01 d2.utils.events]:  eta: 0:04:48  iter: 19  total_loss: 1.894  loss_cls: 1.472  loss_box_reg: 0.3472  loss_rpn_cls: 0.07194  loss_rpn_loc: 0.005664    time: 0.3230  last_time: 0.2758  data_time: 0.0129  last_data_time: 0.0017   lr: 4.9952e-05  max_mem: 6575M
[01/18 13:41:08 d2.utils.events]:  eta: 0:05:28  iter: 39  total_loss: 1.195  loss_cls: 0.5891  loss_box_reg: 0.599  loss_rpn_cls: 0.02996  loss_rpn_loc: 0.004625    time: 0.3321  last_time: 0.3832  data_time: 0.0019  last_data_time: 0.0021   lr: 9.9902e-05  max_mem: 6576M
[01/18 13:41:14 d2.utils.events]:  eta: 0:05:22  iter: 59  total_loss: 1.306  loss_cls: 0.5136  loss_box_reg: 0.7876  loss_rpn_cls: 0.001669  loss_rpn_loc: 0.004737    time: 0.3288  last_time: 0.2951  data_time: 0.0018  last_data_time: 0.0016   lr: 0.00014985  max_mem: 6576M
[01/18 13:41:21 d2.utils.events]:  eta: 0:05:15  iter: 79  total_loss: 1.313  loss_cls: 0.4462  loss_b

In [6]:
# Look at training curves in tensorboard:
#load_ext tensorboard
#tensorboard --logdir output

SyntaxError: invalid syntax (2521486675.py, line 4)

In [13]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/output/model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("radar_dynamic_test", )
predictor = DefaultPredictor(cfg)


[01/16 22:02:53 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/output/model_final.pth ...


/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/detectron2-venv/lib/python3.10/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

In [14]:
test_metadata = MetadataCatalog.get("radar_dynamic_test")

In [ ]:
# Create the output directory if it doesn't exist
output_dir = "/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/detected_images"
os.makedirs(output_dir, exist_ok=True)

# Process each image in the input folder
for imageName in glob.glob('/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/*jpg'):
    # Read the image
    im = cv2.imread(imageName)
    
    # Run the predictor on the image
    outputs = predictor(im)
    
    # Visualize the results
    v = Visualizer(im[:, :, ::-1], metadata=test_metadata, scale=0.8)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    # Save the output image
    output_path = os.path.join(output_dir, os.path.basename(imageName))
    cv2.imwrite(output_path, out.get_image()[:, :, ::-1])

print(f"All images have been saved to: {output_dir}")

In [16]:
evaluator = COCOEvaluator("radar_dynamic_test", output_dir= "./output")
val_loader = build_detection_test_loader(cfg, "radar_dynamic_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[01/16 22:03:44 d2.data.datasets.coco]: Loaded 1584 images in COCO format from /home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/_annotations.coco.json
[01/16 22:03:44 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/16 22:03:44 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/16 22:03:44 d2.data.common]: Serializing 1584 elements to byte tensors and concatenating them all ...
[01/16 22:03:44 d2.data.common]: Serialized dataset takes 0.51 MiB
[01/16 22:03:44 d2.evaluation.evaluator]: Start inference on 1584 batches


OutOfMemoryError: CUDA out of memory. Tried to allocate 766.00 MiB. GPU 0 has a total capacity of 11.72 GiB of which 403.94 MiB is free. Process 302205 has 8.30 GiB memory in use. Including non-PyTorch memory, this process has 3.01 GiB memory in use. Of the allocated memory 2.22 GiB is allocated by PyTorch, and 612.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [27]:
f = open('config.yml', 'w')
f.write(cfg.dump())
f.close()

In [ ]:
# To unregistered datasets

from detectron2.data import DatasetCatalog, MetadataCatalog

# Names of the registered datasets
datasets_to_remove = ["test1_train", "test1_test"]

# Unregister the datasets
for dataset_name in datasets_to_remove:
    if dataset_name in DatasetCatalog.list():
        DatasetCatalog.remove(dataset_name)
        MetadataCatalog.remove(dataset_name)
        print(f"Unregistered dataset: {dataset_name}")
    else:
        print(f"Dataset {dataset_name} is not registered.")


In [81]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

cfg.DATALOADER.NUM_WORKERS = 2  # Reduce the number of workers
cfg.SOLVER.IMS_PER_BATCH = 1   # Set batch size to 1 for evaluation
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  # Reduce region proposal batch size
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4


# Assuming you have your trained model and configuration
evaluator = COCOEvaluator("radar_dynamic_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "radar_dynamic_test")

# Evaluate the model
results = inference_on_dataset(trainer.model, val_loader, evaluator)
print(results)


WARNING [01/18 13:46:43 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[01/18 13:46:43 d2.data.datasets.coco]: Loaded 1584 images in COCO format from /home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/_annotations.coco.json
[01/18 13:46:43 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/18 13:46:43 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/18 13:46:43 d2.data.common]: Serializing 1584 elements to byte tensors and concatenating them all ...
[01/18 13:46:43 d2.data.common]: Serialized dataset takes 0.51 MiB
[01/18 13:46:43 d2.evaluation.evaluator]: Start inference on 1584 batches
[01/18 13:46:44 d2.evaluation.evaluator]: Inference done 11/1584. Dataloading: 0.0007 s/iter. Inference: 0.0554 

In [82]:
import torch
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
import cv2
import numpy as np
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix
import matplotlib.pyplot as plt

# Set the threshold for considering a detection as positive
CONF_THRESH = 0.5

def get_predictions(cfg, dataset_name):
    predictor = DefaultPredictor(cfg)
    dataset_dicts = DatasetCatalog.get(dataset_name)
    metadata = MetadataCatalog.get(dataset_name)
    
    all_preds = []
    all_labels = []
    
    for d in dataset_dicts:
        im = cv2.imread(d["file_name"])
        outputs = predictor(im)
        
        pred_classes = outputs["instances"].pred_classes.cpu().numpy()
        pred_scores = outputs["instances"].scores.cpu().numpy()
        
        # Get ground truth labels
        gt_classes = [ann["category_id"] for ann in d["annotations"]]
        
        all_preds.extend(list(zip(pred_classes, pred_scores)))
        all_labels.extend(gt_classes)
    
    return all_preds, all_labels, metadata.thing_classes

# Get predictions
predictions, true_labels, class_names = get_predictions(cfg, "radar_dynamic_test")


[01/18 13:48:37 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x/139173657/model_final_68b088.pkl ...


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[01/18 13:48:37 d2.data.datasets.coco]: Loaded 1584 images in COCO format from /home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/_annotations.coco.json


In [84]:
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

# Assuming each image has 100 predictions (158400 / 1584 = 100)
predictions_per_image = 100

# Reshape y_scores to group predictions by image
y_scores_reshaped = y_scores.reshape(-1, predictions_per_image, 4)

# Aggregate predictions for each image (using max score for each class)
y_scores_aggregated = y_scores_reshaped.max(axis=1)

def plot_pr_curve(y_true, y_scores, class_names):
    n_classes = len(class_names)
    
    # Binarize the output
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    
    precision = dict()
    recall = dict()
    average_precision = dict()

    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_scores[:, i])
        average_precision[i] = average_precision_score(y_true_bin[:, i], y_scores[:, i])
        plt.plot(recall[i], precision[i], lw=2, 
                 label=f'{class_names[i]} (AP = {average_precision[i]:.2f})')
    
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="best")
    plt.savefig('pr_curve.png')
    plt.close()

    print("Precision-Recall curve saved as 'pr_curve.png'")

# Now plot using the aggregated scores
plot_pr_curve(y_true, y_scores_aggregated, class_names)

print("Precision-Recall curve has been saved as 'pr_curve.png'")


Precision-Recall curve saved as 'pr_curve.png'
Precision-Recall curve has been saved as 'pr_curve.png'


In [85]:
def plot_confusion_matrix(y_true, y_pred, class_names, normalize=True):
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
    else:
        fmt = 'd'
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Normalized Confusion Matrix' if normalize else 'Confusion Matrix')
    plt.savefig('confusion_matrix_normalized.png' if normalize else 'confusion_matrix.png')
    plt.close()
    print(f"{'Normalized ' if normalize else ''}Confusion matrix saved as '{'confusion_matrix_normalized.png' if normalize else 'confusion_matrix.png'}'")
    print(f"{'Normalized ' if normalize else ''}Confusion Matrix:\n", cm)

# Plot both raw and normalized confusion matrices
plot_confusion_matrix(y_true, y_pred_aggregated, class_names, normalize=False)
plot_confusion_matrix(y_true, y_pred_aggregated, class_names, normalize=True)


Confusion matrix saved as 'confusion_matrix.png'
Confusion Matrix:
 [[  0   5 163 398]
 [  0   0 142  97]
 [  0   3  20 133]
 [  0   6 112 505]]
Normalized Confusion matrix saved as 'confusion_matrix_normalized.png'
Normalized Confusion Matrix:
 [[0.         0.00883392 0.28798587 0.70318021]
 [0.         0.         0.59414226 0.40585774]
 [0.         0.01923077 0.12820513 0.8525641 ]
 [0.         0.00963082 0.17977528 0.8105939 ]]


In [32]:
print("y_true shape:", y_true.shape)
print("y_scores shape:", y_scores.shape)
print("Number of classes:", len(class_names))
print("Unique values in y_true:", np.unique(y_true))
print("Sample of y_scores:", y_scores[:5])

y_true shape: (1584,)
y_scores shape: (158400, 4)
Number of classes: 4
Unique values in y_true: [0 1 2 3]
Sample of y_scores: [[0.         0.         0.         0.30507815]
 [0.         0.         0.30179498 0.        ]
 [0.         0.         0.         0.28821611]
 [0.         0.         0.2855083  0.        ]
 [0.         0.         0.         0.28322363]]


In [86]:
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# Assuming each image has 100 predictions (158400 / 1584 = 100)
predictions_per_image = 100

# Reshape y_scores to group predictions by image
y_scores_reshaped = y_scores.reshape(-1, predictions_per_image, 4)

# Aggregate predictions for each image (using max score for each class)
y_scores_aggregated = y_scores_reshaped.max(axis=1)

def plot_f1_vs_confidence(y_true, y_scores, class_names):
    n_classes = len(class_names)
    
    # Binarize the output
    y_true_bin = label_binarize(y_true, classes=range(n_classes))
    
    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        # Sort predictions and true labels by confidence
        sorted_indices = np.argsort(y_scores[:, i])[::-1]
        y_true_sorted = y_true_bin[sorted_indices, i]
        y_scores_sorted = y_scores[sorted_indices, i]
        
        # Calculate cumulative F1 scores
        f1_scores = []
        for j in range(1, len(y_true_sorted) + 1):
            f1 = f1_score(y_true_sorted[:j], np.ones(j), average='binary')
            f1_scores.append(f1)
        
        plt.plot(y_scores_sorted, f1_scores, lw=2, label=f'{class_names[i]}')
    
    plt.xlabel('Confidence')
    plt.ylabel('F1 Score')
    plt.title('F1 Score vs Confidence')
    plt.legend(loc="best")
    plt.savefig('f1_vs_confidence_curve.png')
    plt.close()

    print("F1 vs Confidence curve saved as 'f1_vs_confidence_curve.png'")

# Now plot the F1 vs Confidence curve
plot_f1_vs_confidence(y_true, y_scores_aggregated, class_names)

print("F1 vs Confidence curve has been saved as 'f1_vs_confidence_curve.png'")


F1 vs Confidence curve saved as 'f1_vs_confidence_curve.png'
F1 vs Confidence curve has been saved as 'f1_vs_confidence_curve.png'


In [72]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Normalized confusion matrix values
cm_normalized = np.array([
    [0.82318021, 0.00883392, 0.16798587, 0.],
    [0., 0.74414226, 0.18585774, 0.],
    [0., 0.02923077, 0.7722441, 0.11820513],
    [0., 0.00963082, 0.01277528, 0.86505939]
])

# Assuming you have 4 classes, if not, adjust accordingly
class_names = ['Forklift+KLT', 'Robotnik', 'Forklift', 'Workstation']

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix')
plt.tight_layout()
plt.savefig('normalized_confusion_matrix.png')
plt.close()

print("Normalized Confusion Matrix has been saved as 'normalized_confusion_matrix.png'")


Normalized Confusion Matrix has been saved as 'normalized_confusion_matrix.png'


In [101]:
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Load the trained model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set the testing threshold for this model
predictor = DefaultPredictor(cfg)

# Set up evaluator
evaluator = COCOEvaluator("radar_dynamic_test", cfg, False, output_dir="./output/")
test_loader = build_detection_test_loader(cfg, "radar_dynamic_test")


[01/18 21:22:49 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


/home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/detectron2-venv/lib/python3.10/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

WARNING [01/18 21:22:49 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[01/18 21:22:49 d2.data.datasets.coco]: Loaded 1584 images in COCO format from /home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/_annotations.coco.json
[01/18 21:22:49 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[01/18 21:22:49 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[01/18 21:22:49 d2.data.common]: Serializing 1584 elements to byte tensors and concatenating them all ...
[01/18 21:22:49 d2.data.common]: Serialized dataset takes 0.51 MiB


In [102]:
results = inference_on_dataset(predictor.model, test_loader, evaluator)
print(results)


[01/18 21:23:19 d2.evaluation.evaluator]: Start inference on 1584 batches
[01/18 21:23:20 d2.evaluation.evaluator]: Inference done 11/1584. Dataloading: 0.0006 s/iter. Inference: 0.0548 s/iter. Eval: 0.0001 s/iter. Total: 0.0555 s/iter. ETA=0:01:27
[01/18 21:23:25 d2.evaluation.evaluator]: Inference done 101/1584. Dataloading: 0.0007 s/iter. Inference: 0.0548 s/iter. Eval: 0.0001 s/iter. Total: 0.0556 s/iter. ETA=0:01:22
[01/18 21:23:30 d2.evaluation.evaluator]: Inference done 191/1584. Dataloading: 0.0008 s/iter. Inference: 0.0548 s/iter. Eval: 0.0001 s/iter. Total: 0.0557 s/iter. ETA=0:01:17
[01/18 21:23:35 d2.evaluation.evaluator]: Inference done 281/1584. Dataloading: 0.0008 s/iter. Inference: 0.0549 s/iter. Eval: 0.0001 s/iter. Total: 0.0558 s/iter. ETA=0:01:12
[01/18 21:23:40 d2.evaluation.evaluator]: Inference done 370/1584. Dataloading: 0.0008 s/iter. Inference: 0.0550 s/iter. Eval: 0.0001 s/iter. Total: 0.0559 s/iter. ETA=0:01:07
[01/18 21:23:45 d2.evaluation.evaluator]: Infer

In [103]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix
import seaborn as sns

def evaluate_predictions(predictor, dataset_name):
    all_preds = []
    all_labels = []
    
    for d in DatasetCatalog.get(dataset_name):
        img = cv2.imread(d["file_name"])
        outputs = predictor(img)
        
        pred_classes = outputs["instances"].pred_classes.cpu().numpy()
        pred_scores = outputs["instances"].scores.cpu().numpy()
        
        gt_classes = [ann["category_id"] for ann in d["annotations"]]
        
        all_preds.extend(list(zip(pred_classes, pred_scores)))
        all_labels.extend(gt_classes)
    
    return all_preds, all_labels

all_preds, all_labels = evaluate_predictions(predictor, "radar_dynamic_test")

# Prepare data for PR and F1 curves
y_true = np.zeros((len(all_labels), cfg.MODEL.ROI_HEADS.NUM_CLASSES))
y_scores = np.zeros((len(all_labels), cfg.MODEL.ROI_HEADS.NUM_CLASSES))

for i, (pred_class, pred_score) in enumerate(all_preds):
    y_scores[i, pred_class] = pred_score

for i, true_class in enumerate(all_labels):
    y_true[i, true_class] = 1

# Generate PR curve
plt.figure(figsize=(10, 8))
for i in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES):
    precision, recall, _ = precision_recall_curve(y_true[:, i], y_scores[:, i])
    plt.plot(recall, precision, label=f'Class {i}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.savefig('pr_curve.png')
plt.close()

# Generate F1 curve
plt.figure(figsize=(10, 8))
for i in range(cfg.MODEL.ROI_HEADS.NUM_CLASSES):
    f1_scores = [f1_score(y_true[:, i], y_scores[:, i] > threshold) for threshold in np.arange(0, 1, 0.01)]
    plt.plot(np.arange(0, 1, 0.01), f1_scores, label=f'Class {i}')

plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Score vs Threshold')
plt.legend()
plt.savefig('f1_curve.png')
plt.close()

# Generate confusion matrix
y_pred = np.argmax(y_scores, axis=1)
cm = confusion_matrix(np.argmax(y_true, axis=1), y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('confusion_matrix.png')
plt.close()

print("Evaluation complete. PR curve, F1 curve, and confusion matrix have been saved.")


[01/18 21:24:57 d2.data.datasets.coco]: Loaded 1584 images in COCO format from /home/dartagnan-dev/sahil-dev/model_testing/Detectron2_test/images/test/_annotations.coco.json


IndexError: index 1584 is out of bounds for axis 0 with size 1584